# Print the sensitivity analysis problems

In this example, we show how to print all the sensitivity analysis problems.

In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

When we estimate Sobol' indices, we may encounter the following warning messages:
```
WRN - The estimated first order Sobol index (2) is greater than its total order index . You may increase the sampling size.
WRN - The estimated total order Sobol index (2) is lesser than first order index . You may increase the sampling size.
```
Lots of these messages are printed in the current Notebook. This is why we disable them with:

In [2]:
ot.Log.Show(ot.Log.NONE)

We import the list of problems.

In [3]:
benchmarkProblemList = otb.SensitivityBenchmarkProblemList()
numberOfProblems = len(benchmarkProblemList)
numberOfProblems

11

For each problem in the benchmark, print the problem name and the exact indices.

In [4]:
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    first_order_indices = problem.getFirstOrderIndices()
    total_order_indices = problem.getTotalOrderIndices()
    dimension = problem.getInputDistribution().getDimension()
    print(
        "#",
        i,
        ":",
        name,
        " : S = ",
        first_order_indices,
        "T=",
        total_order_indices,
        ", dimension=",
        dimension,
    )

# 0 : GaussianSum  : S =  [0.5,0.5] T= [0.5,0.5] , dimension= 2
# 1 : GaussianProduct  : S =  [0,0] T= [1,1] , dimension= 2
# 2 : GSobol  : S =  [0.986712,0.00986712,9.86712e-05] T= [0.990034,0.0131566,0.000132] , dimension= 3
# 3 : Ishigami  : S =  [0.313905,0.442411,0] T= [0.557589,0.442411,0.243684] , dimension= 3
# 4 : Borehole  : S =  [0.66,0,0,0.09,0,0.09,0.09,0.02] T= [0.69,0,0,0.11,0,0.11,0.1,0.02] , dimension= 8
# 5 : Dirichlet  : S =  [0.525547,0.233577,0.131387] T= [0.620438,0.310219,0.182482] , dimension= 3
# 6 : Flooding  : S =  [0.38,0.13,0.25,0,0.19,0.02,0,0] T= [0.4,0.15,0.25,0.01,0.19,0.02,0,0] , dimension= 8
# 7 : Morris  : S =  [0.08,0.08,0.06,0.08,0.06,0.13,0.06,0.13,0.13,0.12,0,0,0,0,0,0,0,0,0,0]#20 T= [0.11,0.11,0.06,0.11,0.06,0.13,0.06,0.13,0.13,0.12,0,0,0,0,0,0,0,0,0,0]#20 , dimension= 20
# 8 : N.L. Oscillator  : S =  [0.4,0.03,0.09,0.18,0.12,0.05,0.05,0] T= [0.4,0.04,0.1,0.23,0.16,0.07,0.06,0.01] , dimension= 8
# 9 : Borgonovo  : S =  [0.157895,0.157895,0.63157

In [5]:
problem_names = [
    benchmarkProblem.getName() for benchmarkProblem in benchmarkProblemList
]
columns = ["$d$"]
df_problems_list = pd.DataFrame(index=problem_names, columns=columns)
for problem in benchmarkProblemList:
    name = problem.getName()
    d = problem.getInputDistribution().getDimension()
    df_problems_list.loc[name] = [int(d)]
df_problems_list

,$d$
GaussianSum,2
GaussianProduct,2
GSobol,3
Ishigami,3
Borehole,8
Dirichlet,3
Flooding,8
Morris,20
N.L. Oscillator,8
Borgonovo,3


## Use Borgonovo problem

In [6]:
problem = otb.BorgonovoSensitivity()
distribution = problem.getInputDistribution()
model = problem.getFunction()

In [7]:
# Exact first and total order
exact_first_order = problem.getFirstOrderIndices()
exact_total_order = problem.getTotalOrderIndices()

## Saltelli estimator with Monte-Carlo sample

In [8]:
sample_size = 10000

In [9]:
inputDesign = ot.SobolIndicesExperiment(distribution, sample_size).generate()
outputDesign = model(inputDesign)

In [10]:
# Compute first order indices using the Saltelli estimator
sensitivityAnalysis = ot.SaltelliSensitivityAlgorithm(
    inputDesign, outputDesign, sample_size
)
computed_first_order = sensitivityAnalysis.getFirstOrderIndices()
computed_total_order = sensitivityAnalysis.getTotalOrderIndices()

In [11]:
# Compare with exact results
print("Sample size : ", sample_size)
# First order
print("Computed first order = ", computed_first_order)
print("Exact first order = ", exact_first_order)
# Total order
print("Computed total order = ", computed_total_order)
print("Exact total order = ", exact_total_order)

Sample size :  10000
Computed first order =  [0.166136,0.157381,0.623798]
Exact first order =  [0.157895,0.157895,0.631579]
Computed total order =  [0.207054,0.2159,0.623765]
Exact total order =  [0.210526,0.210526,0.631579]


## Saltelli estimator with Quasi Monte-Carlo sample

In [12]:
sample_size = 500

In [13]:
dimension = distribution.getDimension()
sequence = ot.SobolSequence(dimension)
restart = True
experiment = ot.LowDiscrepancyExperiment(sequence, distribution, sample_size, restart)

In [14]:
inputDesign = ot.SobolIndicesExperiment(experiment).generate()
outputDesign = model(inputDesign)

In [15]:
# Compute first order indices using the Saltelli estimator
sensitivityAnalysis = ot.SaltelliSensitivityAlgorithm(
    inputDesign, outputDesign, sample_size
)
first_order = sensitivityAnalysis.getFirstOrderIndices()
total_order = sensitivityAnalysis.getTotalOrderIndices()

In [16]:
# Compare with exact results
print("Sample size : ", sample_size)
# First order
print("Computed first order = ", computed_first_order)
print("Exact first order = ", exact_first_order)
# Total order
print("Computed total order = ", computed_total_order)
print("Exact total order = ", exact_total_order)

Sample size :  500
Computed first order =  [0.166136,0.157381,0.623798]
Exact first order =  [0.157895,0.157895,0.631579]
Computed total order =  [0.207054,0.2159,0.623765]
Exact total order =  [0.210526,0.210526,0.631579]


## Loop over the estimators

In [17]:
print("Available estimators:")
estimators_list = otb.SensitivityBenchmarkMetaAlgorithm.GetEstimators()
for sobolAlgorithm in estimators_list:
    name = sobolAlgorithm.getClassName()
    print(" - ", name)

Available estimators:
 -  SaltelliSensitivityAlgorithm
 -  MartinezSensitivityAlgorithm
 -  JansenSensitivityAlgorithm
 -  MauntzKucherenkoSensitivityAlgorithm


In [18]:
metaSAAlgorithm = otb.SensitivityBenchmarkMetaAlgorithm(problem)

In [19]:
print("Monte-Carlo sampling")
for sobolAlgorithm in estimators_list:
    (
        computed_first_order,
        computed_total_order,
    ) = metaSAAlgorithm.runSamplingEstimator(sample_size)
    name = sobolAlgorithm.getClassName()
    print(name)
    print("    S = ", computed_first_order)
    print("    T = ", computed_total_order)

Monte-Carlo sampling
SaltelliSensitivityAlgorithm
    S =  [0.146135,0.139029,0.657612]
    T =  [0.224719,0.240232,0.642326]
MartinezSensitivityAlgorithm
    S =  [0.153514,0.139747,0.66382]
    T =  [0.251203,0.263947,0.597202]
JansenSensitivityAlgorithm
    S =  [0.1084,0.114513,0.445419]
    T =  [0.262191,0.219558,0.592405]
MauntzKucherenkoSensitivityAlgorithm
    S =  [0.132499,0.172125,0.610951]
    T =  [0.130792,0.196478,0.726522]


In [20]:
print("Quasi Monte-Carlo sampling")
for estimator in ["Saltelli", "Martinez", "Jansen", "MauntzKucherenko"]:
    (
        computed_first_order,
        computed_total_order,
    ) = metaSAAlgorithm.runSamplingEstimator(
        sample_size, estimator=estimator, sampling_method="QMC"
    )
    name = sobolAlgorithm.getClassName()
    print(name)
    print("    S = ", computed_first_order)
    print("    T = ", computed_total_order)

Quasi Monte-Carlo sampling
MauntzKucherenkoSensitivityAlgorithm
    S =  [0.147144,0.151326,0.624455]
    T =  [0.212622,0.209087,0.643622]
MauntzKucherenkoSensitivityAlgorithm
    S =  [0.147601,0.150976,0.629051]
    T =  [0.210037,0.210799,0.640943]
MauntzKucherenkoSensitivityAlgorithm
    S =  [0.151099,0.149846,0.632098]
    T =  [0.20915,0.21105,0.635553]
MauntzKucherenkoSensitivityAlgorithm
    S =  [0.159027,0.163209,0.636338]
    T =  [0.212622,0.209087,0.643622]


In [21]:
print("Polynomial chaos")
sample_size = 500
(
    computed_first_order,
    computed_total_order,
) = metaSAAlgorithm.runPolynomialChaosEstimator(
    sample_size_train=sample_size,
    sample_size_test=2,
    total_degree=5,
    hyperbolic_quasinorm=0.5,
)
print("    S = ", computed_first_order)
print("    T = ", computed_total_order)

Polynomial chaos
    S =  [0.157895,0.157895,0.631579]
    T =  [0.210526,0.210526,0.631579]


## Define the metric

We consider the following accuracy metrics:
* the vector or log relative errors for a given index (first order or total order),
* the mean log relative error, as the mean of the LRE vector (first order or total order),
* the average mean log relative error, as the mean of the first and total order mean log relative error.

Larger LRE values are prefered.

The first order (resp. total order) mean LRE represents the mean number of digits for all components of the first order indices (resp. total order indices). The average mean LRE represents the mean LRE for both first and total order indices.

In [22]:
S_LRE = ot.Point(dimension)
T_LRE = ot.Point(dimension)
for i in range(dimension):
    S_LRE[i] = otb.ComputeLogRelativeError(
        computed_first_order[i], exact_first_order[i]
    )
    T_LRE[i] = otb.ComputeLogRelativeError(
        computed_total_order[i], exact_total_order[i]
    )

In [23]:
print("LRE S = ", S_LRE)
print("LRE T = ", T_LRE)

LRE S =  [15.153,15.6536,15.454]
LRE T =  [15.6536,15.2779,15.454]


In [24]:
mean_LRE_S = sum(S_LRE) / dimension
mean_LRE_T = sum(T_LRE) / dimension
mean_LRE = (mean_LRE_S + mean_LRE_T) / 2.0
print("Mean LRE S = %.2f" % (mean_LRE_S))
print("Mean LRE T = %.2f" % (mean_LRE_T))
print("Mean LRE = %.2f" % (mean_LRE))

Mean LRE S = 15.42
Mean LRE T = 15.46
Mean LRE = 15.44


The digit per point ratio measure the number of digits relatively to the sample size. A greater value is prefered.

In [25]:
digit_per_point_ratio = mean_LRE / sample_size
print("Digit / point = %.3e" % (digit_per_point_ratio))

Digit / point = 3.088e-02
